In [3]:
import xgboost as xgb
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12,4

In [4]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sub = pd.read_csv('Sub.csv')
labels = pd.read_csv('Labels.csv')

In [5]:
labels.status_group.value_counts()
train['data'] ='train'
test['data'] = 'test'

In [6]:
combi = pd.concat([train,test])

In [28]:
print combi.shape
print train.shape
print test.shape

(74250, 42)
(59400, 41)
(14850, 41)


In [35]:
combi.dtypes

id                         int64
amount_tsh               float64
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_type              object
water_qual

In [15]:
combi.apply(lambda x: sum(x.isnull()))

id                           0
amount_tsh                   0
date_recorded                0
funder                    4504
gps_height                   0
installer                 4532
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 470
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting               0
recorded_by                  0
scheme_management            0
scheme_name              35258
permit                       0
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [49]:
combi.funder.value_counts()
combi.installer.value_counts()
combi.wpt_name.value_counts()
combi.basin.value_counts()
combi.subvillage.value_counts()
combi.region.value_counts()
combi.region_code.value_counts()
combi.district_code.value_counts()
combi.extraction_type_class.value_counts()
combi.management_group.value_counts()
combi.payment_type.value_counts()
combi.quality_group.value_counts()
combi.quantity_group.value_counts()
combi.source_type.value_counts()
combi.source_class.value_counts()
combi.waterpoint_type.value_counts()
combi.waterpoint_type_group.value_counts()
combi.source.value_counts()
combi.quantity.value_counts()
combi.water_quality.value_counts()
combi.payment.value_counts()
combi.permit.value_counts()
combi.management.value_counts()
combi.construction_year.value_counts()
combi.scheme_management.value_counts()
combi.recorded_by.value_counts()
combi.lga.value_counts()
combi.ward.value_counts()
combi.population.value_counts()

spring                  21216
shallow well            21140
machine dbh             13822
river                   11964
rainwater harvesting     2863
hand dtw                 1108
lake                      950
dam                       840
other                     261
unknown                    86
Name: source, dtype: int64

In [51]:
num_variables = ['id','amount_tsh', 'age2','age1','gps_height','year_recorded', 'num_private','construction_year','population','latitude','longitude','data']
cat_variables = ['month_recorded','day_recorded','funder','basin','region','region_code','district_code','extraction_type_class','management_group','payment_type','quality_group',
                'quantity_group','source','source_type','source_class','waterpoint_type','waterpoint_type_group','quantity','water_quality',
                 'payment','permit','management','extraction_type_group','extraction_type','scheme_management','public_meeting']
tot_variables = num_variables + cat_variables

In [50]:
combi.permit = combi.permit.fillna('NA')
combi.scheme_management = combi.scheme_management.fillna('NA')
combi.public_meeting = combi.public_meeting.fillna('NA')
combi.date_recorded = pd.to_datetime(combi.date_recorded)
combi['year_recorded'] = combi.date_recorded.map(lambda x : x.year)
combi['month_recorded'] = combi.date_recorded.map(lambda x : x.month)
combi['day_recorded'] = combi.date_recorded.map(lambda x : x.dayofweek)
combi.drop('date_recorded',axis=1,inplace=True)
combi.funder = combi.funder.fillna('NA')
combi.funder = np.where(combi.funder == "Government of Tanzania", 'Govt', np.where(combi.funder == "NA","NA",
                                                                                   "others"))
combi.installer = combi.installer.fillna('NA')
combi.installer = np.where(combi.installer == "DWE", 'DWE', np.where(combi.installer == "NA","NA",
                                                                                   "others"))
combi['age1'] = 2017 - combi['construction_year']
combi['age2'] = combi['year_recorded'] - combi['construction_year']

In [52]:
combi_new = combi[tot_variables]


In [53]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
var_to_encode = cat_variables
for col in var_to_encode:
    combi_new[col] = le.fit_transform(combi_new[col])
combi_new = pd.get_dummies(combi_new, columns=var_to_encode)

/home/anoop/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [54]:
train_new = combi_new[combi_new.data == 'train']
test_new = combi_new[combi_new.data == 'test']

In [55]:
train_new = pd.merge(train_new,labels,on = 'id')

In [56]:
target = 'status_group' 

In [57]:
from xgboost.sklearn import XGBClassifier
xgb1 = XGBClassifier(
 learning_rate =0.01,
 n_estimators=1000,
 max_depth=20,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softmax',
 nthread=4,
 scale_pos_weight=1,
 seed=27)


In [58]:
from sklearn.model_selection import train_test_split
predictors = [x for x in combi_new.columns if x not in [target, 'id','data']]
train_set,test_set = train_test_split(train_new, test_size=0.2, random_state=42)

In [75]:
print train_new.shape
print train_set.shape
print test_set.shape

(59400, 195)
(47520, 195)
(11880, 195)


In [59]:
xgb1.fit(train_new[predictors],train_new[target])

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=20,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=4,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8)

In [77]:
test_set['prediction'] = xgb1.predict(test_set[predictors])


/home/anoop/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [60]:
print xgb1.score(test_set[predictors], test_set[target])
test_new['status_group'] = xgb1.predict(test_new[predictors])


0.98063973064


In [62]:
sub1 = test_new[['id','status_group']]
sub1.to_csv('sub14.csv',index=False)

/home/anoop/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
